In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Define trading parameters
symbol = 'BTCUSDT'
interval = '15m'
initial_balance = 0.4  # Starting with $1
profit_target = 100  # Target $10 profit


def get_historical_klines(symbol, interval, limit=1000):
    klines = client.futures_klines(symbol=symbol, interval=interval, limit=limit)
    df = pd.DataFrame(klines, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_asset_volume', 'number_of_trades', 'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'])
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    return df

# Function to calculate simple moving average (SMA)
def calculate_sma(df, window=20):
    df['close'] = pd.to_numeric(df['close'], errors='coerce')  # Convert 'close' to numeric
    df['sma'] = df['close'].rolling(window=window).mean()
    df = df.dropna()  # Drop rows with missing values
    return df

# Function to simulate a market order to buy
def simulate_buy(price, balance):
    quantity = balance / price
    return quantity

# Function to simulate a market order to sell
def simulate_sell(price, quantity):
    return quantity * price

# Function to backtest the trading strategy
def backtest_strategy(data):
    balance = initial_balance
    in_position = False
    profit = 0
    num_trades = 0  # Initialize the number of trades counter

    # Initialize empty lists for buy and sell signals
    buy_signals = []
    sell_signals = []

    for i in range(20, len(data)):
        if data['close'][i] > data['sma'][i] and not in_position:
            # Buy signal
            buy_price = float(data['close'][i])
            in_position = True

            # Calculate the quantity based on available balance
            quantity = simulate_buy(buy_price, balance)
            balance -= quantity * buy_price

            # Append the buy signal to the list
            buy_signals.append(data['timestamp'][i])

            # Increment the number of trades
            num_trades += 1

        elif data['close'][i] < data['sma'][i] and in_position:
            # Sell signal
            sell_price = float(data['close'][i])
            in_position = False

            # Calculate the profit from the sale
            profit += simulate_sell(sell_price, quantity)
            balance += profit

            # Check if profit target is reached
            if balance >= initial_balance + profit_target:
                break

            # Append the sell signal to the list
            sell_signals.append(data['timestamp'][i])

            # Increment the number of trades
            num_trades += 1

    # Calculate final balance including any remaining holdings
    final_balance = balance + simulate_sell(float(data['close'].iloc[-1]), quantity)

    # Add 'buy_signal' and 'sell_signal' columns to the DataFrame
    data['buy_signal'] = data['timestamp'].isin(buy_signals).astype(int)
    data['sell_signal'] = data['timestamp'].isin(sell_signals).astype(int)

    # Extract start and end dates
    start_date = data['timestamp'].iloc[20]
    end_date = data['timestamp'].iloc[-1]

    return final_balance, num_trades, start_date, end_date


symbol = 'BTCUSDT'
interval = '15m'
# Load historical data for backtesting
historical_data = get_historical_klines(symbol, interval)  # Replace with your historical data file

# Calculate moving averages
historical_data = calculate_sma(historical_data)


# Perform backtest
final_balance, num_trades, start_date, end_date = backtest_strategy(historical_data)

print(f"Initial Balance: ${initial_balance}")
print(f"Final Balance: ${final_balance}")
print(f"Profit: ${final_balance - initial_balance}")
print(f"Trades: {num_trades}")

# Calculate the number of days
num_days = (end_date - start_date).days
print(f"Number of Days: {num_days}")


Initial Balance: $0.4
Final Balance: $158.5783945807959
Profit: $158.1783945807959
Trades: 17
Number of Days: 10
